# Term Deposit Marketing Campaign Analysis

The aim of the marketing campaign was to get customers to subscribe to a bank term deposit product. Whether they did this or not is variable ‘y’ in the data set. The bank in question is considering how to optimise this campaign in future.

What would your recommendations to the marketing manager be?

________

### Step 1: Import Modules & Data

#### Step 1a: Import modules, read tab delimited file and review dataframe

In [ ]:
# import pandas and numpy
import pandas as pd
import numpy as np

# import visualisation modules
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read in tab delimited data file
inp = pd.read_csv('../Data/Bank_Marketing_Downloads.txt', header=0, sep='\t')

# review first five records of inp dataframe
inp.head()

In [ ]:
print('Dataframe contains', len(inp), 'rows and', len(inp.columns), 'columns')

*Observations:*
* Imported dataframe contains 45,211 rows and 17 columns

________

### Step 2: Data Exploration

#### Step 2a: Review data type and check for null values for each column

In [ ]:
# print column data types and check for any columns with null values
inp.info()

In [ ]:
# check for duplicate records
print(inp.duplicated().value_counts())

*Observations:*
* Dataframe (inp) has 7 numerical and 10 categorical columns. 
* None of these columns have any null values. 
* There are no duplicates.

#### Step 2b: Explore numerical columns

In [ ]:
# summary stats for numerical columns
inp.describe()

*Observations:*
* Mean age is approximately 41, with minimum age 18 and maximum age 95.
* Mean balance is $1362. However standard deviation is a high number suggesting balance amount is more spread out and indicating outliers in the dataset.
* 75% of customer were not contacted prior to this campaign. 
* Although duration of call might not be important ie: the longer the call, the more likely customer subscribes to term deposit. However, I am keen to investigate what was the mean call duration of customers who subscribed. More on this a little further down this notebook.


In [ ]:
# distribution of each int columns
plt.style.use('seaborn-whitegrid')

inp.hist(bins=20, figsize=(14,10), color='#e3b18d')
plt.show()

*Observations:*
* We can see that numerical columns have outliers, especially 'campaign', 'pdays' and 'previous' columns), indicating outliers of possibly noisy data (incorrect value).

In [ ]:
# let's take a closer look at 'campaign','pdays','previous'
inp[['balance','duration','campaign','pdays','previous']].describe()

In [ ]:
# Percentage of 'balance' in negative:
len (inp[inp['balance'] < 0] ) / len(inp) * 100

In [ ]:
# Percentage of 'balance' above 10000:
len (inp[inp['balance'] > 10000] ) / len(inp) * 100

*Observations:*
* 'balance' holds the average yearly balance, in euros (numeric).
* 8.4% of customers have negative average yearly balance.
* Because standard deviation of balance is such a high number, balance above 10000 are noise.
* Suggest to impute them with mean balance values while data cleaning.

In [ ]:
# Percentage of 'duration' less than a minute:
len (inp[inp['duration'] < 60] ) / len(inp) * 100

In [ ]:
# Percentage of 'duration' more than 30mins:
len (inp[inp['duration'] > 1800 ] ) / len(inp) * 100

*Observations:*
* 'duration' holds last contact duration, in seconds (numeric) other attributes
* Numbers for 'duration' less than 60secs (1min) and more than 1800secs (30mins) are noise

In [ ]:
# Percentage of 'campaign' values above 30:
len (inp[inp['campaign'] > 30] ) / len(inp) * 100

*Observations:*
* 'campaign' holds the number of contacts performed during this campaign and for this client (numeric, includes last contact) 
* Numbers for 'campaign' above 30 are clearly noise
* Suggest to impute them with mean campaign values while data cleaning.

In [ ]:
# Percentage of 'pdays' values < 0 ie: customer was not previously contacted:
len (inp[inp['pdays'] < 0] ) / len(inp) * 100

In [ ]:
# Percentage of 'pdays' values above 365 (1 year):
len (inp[inp['pdays'] > 365] ) / len(inp) * 100

*Observations:*
* 'pdays' holds the number of days that passed by after the client was last contacted from a previous campaign Looking closer into 'pdays' data we can see that:
* Almost 82% of customers weren't previously contacted.
* 1.42%% of values are above 365 (1 year). They are possibly outliers, so we should consider imputing something (possibly mean value) instead of these values.

In [ ]:
# Percentage of 'previous' values above 34:
len (inp[inp['previous'] > 34] ) / len(inp) * 100

*Observations:*
* 'previous' holds the number of contacts performed before this campaign and for this client (numeric)
* Numbers for 'previous' above 34 are also really strange
* Suggest to impute them with mean values while data cleaning.

#### Step 2c: Explore categorical columns

In [ ]:
# distribution of categorical columns
colcatg = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month','poutcome', 'y']
plt.style.use('seaborn-whitegrid')

fig, axs = plt.subplots(2, 5, sharex=False, sharey=False, figsize=(30, 15))

counter = 0
for catg in colcatg:
    value_counts = inp[catg].value_counts()
    
    x = counter // 5
    y = counter % 5
    x_pos = np.arange(0, len(value_counts))
    
    axs[x, y].bar(x_pos, value_counts.values, tick_label = value_counts.index, color ='#e3b18d')
    
    axs[x, y].set_title(catg)
    
    for tick in axs[x, y].get_xticklabels():
        tick.set_rotation(90)
    
    counter += 1

plt.show()

In [ ]:
# closer look at variable y, taken up term deposit
total = len(inp['y'])
print(inp['y'].value_counts())
print(inp['y'].value_counts() / total)

In [ ]:
# visualise term deposit subscribers
sizes = list(inp['y'].value_counts().sort_values().values)
labels = list(inp['y'].value_counts().sort_values().index)

explode = [0, 0.1]

plt.figure(figsize=(4,4))
plt.pie(sizes, labels=labels, explode = explode,
        autopct=lambda p:f'{p:.2f}%', colors = ('#b85636','#e8b777'))
plt.title('Term Deposit Subscribers')
plt.tight_layout()
plt.show()

*Observations:*
* Main call out from looking at distribution of categorical columns is the imbalance in 'y' column (subscribed to term deposit)
* 11.7% subscribed, while 88.3% did not subscribe to term deposit

#### Step 2d: Let's compare term deposit (column y) and compare to categorical columns

In [ ]:
# a: month and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['month'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['month'].value_counts()

dfc.plot.bar(title = 'Contact Month and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# b: job and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['job'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['job'].value_counts()

dfc.plot.bar(title = 'Job and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# c: education and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['education'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['education'].value_counts()

dfc.plot.bar(title = 'Education and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# d: marital status and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['marital'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['marital'].value_counts()

dfc.plot.bar(title = 'Marital Status and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# e: credit default and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['default'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['default'].value_counts()

dfc.plot.bar(title = 'Credit Default and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# f: housing loan and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['housing'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['housing'].value_counts()

dfc.plot.bar(title = 'Housing and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# g: personal loan and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['loan'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['loan'].value_counts()

dfc.plot.bar(title = 'Personal Loan and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# h: communication type and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['contact'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['contact'].value_counts()

dfc.plot.bar(title = 'Communication Type and Term Deposit Take Up', color = ('#b85636','#e8b777'))

In [ ]:
# i: poutcome and term deposit take up
dfc = pd.DataFrame()

dfc['yes'] = inp[inp['y'] == 'yes']['poutcome'].value_counts()
dfc['no'] = inp[inp['y'] == 'no']['poutcome'].value_counts()

dfc.plot.bar(title = 'Previous Campaign Outcome and Term Deposit Take Up', color = ('#b85636','#e8b777'))

*Observations:*
* Although the months of Mar, Sept, Oct and Dec had the least amount of leads, difference between yes/no is not significant as other months ie: higher subscription rate of total customers contacted in Mar, Sept, Oct and Dec.
* Customers in management are more likely to take term desposit.
* Those with tertiary education are more likely to take up term deposit
* Married customers are less likely to take up term deposit.
* Those who don't don't have a credit default, housing or personal loan are more likely to take up term deposit.
* Communication type cellular are more likely to take up term deposit.
* Customers where there was a positive outcome in previous campaign are highly more likely to take up term deposit. 

### Step 2e: Let's compare term deposit (column y) and compare to numerical columns

In [ ]:
# a: balance and deposit
dfn = pd.DataFrame()
dfn['balance_yes'] = (inp[inp['y'] == 'yes'][['y','balance']].describe())['balance']
dfn['balance_no'] = (inp[inp['y'] == 'no'][['y','balance']].describe())['balance']

dfn

In [ ]:
dfn.drop(['count', '25%', '50%', '75%']).plot.bar(title = 'Balance and Deposit Stats', 
        color = ('#b85636','#e8b777'))

In [ ]:
# b: age and deposit
dfn = pd.DataFrame()
dfn['age_yes'] = (inp[inp['y'] == 'yes'][['y','age']].describe())['age']
dfn['age_no'] = (inp[inp['y'] == 'no'][['y','age']].describe())['age']

dfn

In [ ]:
dfn.drop(['count', '25%', '50%', '75%']).plot.bar(title = 'Age and Deposit Stats', 
        color = ('#b85636','#e8b777'))

In [ ]:
# c: number of contacts performed during this campaign ('campaign') and deposit
dfn = pd.DataFrame()
dfn['campaign_yes'] = (inp[inp['y'] == 'yes'][['y','campaign']].describe())['campaign']
dfn['campaign_no'] = (inp[inp['y'] == 'no'][['y','campaign']].describe())['campaign']

dfn

In [ ]:
dfn.drop(['count', '25%', '50%', '75%']).plot.bar(
        title = 'Number of Contacts this Campaign and Deposit Stats', 
        color = ('#b85636','#e8b777'))

In [ ]:
# d: number of contacts performed during previous campaign ('previous') and deposit
dfn = pd.DataFrame()
dfn['previous_yes'] = (inp[inp['y'] == 'yes'][['y','previous']].describe())['previous']
dfn['previous_no'] = (inp[inp['y'] == 'no'][['y','previous']].describe())['previous']

dfn

In [ ]:
dfn.drop(['count', '25%', '50%', '75%']).plot.bar(
        title = 'Number of Contacts Previous Campaign and Deposit Stats', 
        color = ('#b85636','#e8b777'))

*Observations:*
* People who subscribed for term deposit tend to have greater balance and age values.
* People who subscribed for term deposit tend to have fewer number of contacts during this campaign.

________

### Step 3: Data Cleaning

1. Convert columns with 'yes' and 'no' values to boolean columns;
2. Convert categorical columns into dummy variables.

In [ ]:
def get_dummy_from_bool(row, column_name):
    ''' Returns 0 if value in column_name is no, returns 1 if value in column_name is yes'''
    return 1 if row[column_name] == 'yes' else 0

def get_correct_values(row, column_name, threshold, df):
    ''' Returns mean value if value in column_name is above threshold'''
    if row[column_name] <= threshold:
        return row[column_name]
    else:
        mean = df[df[column_name] <= threshold][column_name].mean()
        return mean

def clean_data(df):
    '''
    INPUT
    df - pandas dataframe containing bank marketing campaign dataset
    
    OUTPUT
    df - cleaned dataset:
    1. columns with 'yes' and 'no' values are converted into boolean variables;
    2. categorical columns are converted into dummy variables;
    3. drop irrelevant columns.
    4. impute incorrect values
    '''
    
    cleaned_df = inp.copy()
    
    #convert columns containing 'yes' and 'no' values to boolean variables and drop original columns
    bool_columns = ['default', 'housing', 'loan', 'y']
    for bool_col in bool_columns:
        cleaned_df[bool_col + '_bool'] = inp.apply(lambda row: get_dummy_from_bool(row, bool_col),axis=1)
    
    cleaned_df = cleaned_df.drop(columns = bool_columns)
    
    #convert categorical columns to dummies
    # cat_columns = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
    cat_columns = ['job', 'marital', 'education', 'contact', 'poutcome']
    
    for col in  cat_columns:
        cleaned_df = pd.concat([cleaned_df.drop(col, axis=1),
                                pd.get_dummies(cleaned_df[col], prefix=col, prefix_sep='_',
                                               drop_first=True, dummy_na=False)], axis=1)
    
    #drop irrelevant columns
    cleaned_df = cleaned_df.drop(columns = ['month'])

    #impute incorrect values and drop original columns
    cleaned_df['balance_cleaned'] = df.apply(lambda row: get_correct_values(row, 'balance', 10000, cleaned_df),axis=1)
    cleaned_df['campaign_cleaned'] = df.apply(lambda row: get_correct_values(row, 'campaign', 30, cleaned_df),axis=1)
    cleaned_df['previous_cleaned'] = df.apply(lambda row: get_correct_values(row, 'previous', 34, cleaned_df),axis=1)
    
    cleaned_df = cleaned_df.drop(columns = ['balance', 'campaign', 'previous'])
    
    return cleaned_df

In [ ]:
# clean the dataset
cleaned_df = clean_data(inp)
cleaned_df.head()

________

### Step 4: ML Classification Model (XGBoost)

Use cleaned datasets for prediction of campaign outcome with help of machine learning classification models. I will use XGBoost, which is one of the most common machine learning libraries for modelling.

Resulting model will also help me understand, which features have the greatest importance for the prediction of the results of the campaign.

In [ ]:
#import machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import xgboost

from sklearn.model_selection import train_test_split #split
from sklearn.metrics import accuracy_score #metrics

#tools for hyperparameters search
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
# Create X and y datasets for training the model and split into train and test datasets.
X = cleaned_df.drop(columns = 'y_bool')
y = cleaned_df[['y_bool']]

In [ ]:
cleaned_df['y_bool'].value_counts()

To work with imbalance data in y_bool column, I will use undersample, oversampling and Synthetic Minority Oversampling Technique (SMOTE)

##### Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
rus = RandomUnderSampler(random_state=0)
X_Usampled, y_Usampled = rus.fit_resample(X, y)
y_Usampled.value_counts()

##### Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler(random_state=0)
X_Osampled, y_Osampled = ros.fit_resample(X, y)
y_Osampled.value_counts()

##### Synthetic Minority Oversampling Technique (SMOTE)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state=0)
X_SMOTE, y_SMOTE = sm.fit_resample(X, y)
y_SMOTE.value_counts()

##### Generate train and test data

In [ ]:
TEST_SIZE = 0.3
RAND_STATE = 42

In [ ]:
# Without Sampling
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state=RAND_STATE)

# Undersampling
# X_train, X_test, y_train, y_test = train_test_split(X_Usampled, y_Usampled, test_size = TEST_SIZE, random_state=RAND_STATE)

# Oversampling
# X_train, X_test, y_train, y_test = train_test_split(X_Osampled, y_Osampled, test_size = TEST_SIZE, random_state=RAND_STATE)

# SMOTE
X_train, X_test, y_train, y_test = train_test_split(X_SMOTE, y_SMOTE, test_size = TEST_SIZE, random_state=RAND_STATE)

In [ ]:
#train XGBoost model
xgb = xgboost.XGBClassifier(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train.squeeze().values)

#calculate and print scores for the model for top 15 features
y_train_preds = xgb.predict(X_train)
y_test_preds = xgb.predict(X_test)

print('XGB accuracy score for train: %.3f: test: %.3f' % (
        accuracy_score(y_train, y_train_preds),
        accuracy_score(y_test, y_test_preds)))

*Observations:*
* Without sampling: XGB accuracy score for train: 0.932: test: 0.901
* Undersampling: XGB accuracy score for train: 0.899: test: 0.830
* Oversampling: XGB accuracy score for train: 0.885: test: 0.872
* SMOTE: XGB accuracy score for train: 0.946: test: 0.929

SMOTE gives the highest accuracy score for train and test. 

In [ ]:
# get feature importances from the model
headers = ["name", "score"]
values = sorted(zip(X_train.columns, xgb.feature_importances_), key=lambda x: x[1] * -1)
xgb_feature_importances = pd.DataFrame(values, columns = headers)

# plot feature importances
x_pos = np.arange(0, len(xgb_feature_importances))
plt.bar(x_pos, xgb_feature_importances['score'], color = '#e3b18d')
plt.xticks(x_pos, xgb_feature_importances['name'])
plt.xticks(rotation=90)
plt.title('Feature importance (XGB)')

plt.show()

In [ ]:
# Find out account balance, which marketing campaign should focus on:
df_new = cleaned_df.copy()

# introduce new column 'balance_buckets'
df_new['balance_buckets'] = pd.qcut(df_new['balance_cleaned'], 50, labels=False, duplicates = 'drop')

# group by 'balance_buckets' and find average campaign outcome per balance bucket
mean_deposit = df_new.groupby(['balance_buckets'])['y_bool'].mean()

# plot
plt.plot(mean_deposit.index, mean_deposit.values, color = '#e3b18d')
plt.title('Mean % Subscription Depending on Account Balance')
plt.xlabel('balance bucket')
plt.ylabel('% subscription')
plt.show()

In [ ]:
df_new[df_new['balance_buckets'] == 34]['balance_cleaned'].min()

In [ ]:
# introduce new column 'age_buckets'
df_new['age_buckets'] = pd.qcut(df_new['age'], 20, labels=False, duplicates = 'drop')

# group by 'age_buckets' and find average campaign outcome per age bucket
mean_age = df_new.groupby(['age_buckets'])['y_bool'].mean()

# plot
plt.plot(mean_age.index, mean_age.values, color = '#e3b18d')
plt.title('Mean % subscription depending on age')
plt.xlabel('age bucket')
plt.ylabel('% subscription')
plt.show()

In [ ]:
df_new[df_new['age_buckets'] == 3]['age'].max()

In [ ]:
df_new[df_new['age_buckets'] == 17]['age'].min()

In [ ]:
# Find out appropriate number of contacts with the customer during campaign:
# introduce new column 'campaign_buckets'
df_new['campaign_buckets'] = pd.qcut(df_new['campaign_cleaned'], 20, labels=False, duplicates = 'drop')

# group by 'campaign_buckets' and find average campaign outcome per campaign bucket
mean_campaign = df_new.groupby(['campaign_buckets'])['y_bool'].mean()

# plot average campaign outcome per bucket 
plt.plot(mean_campaign.index, mean_campaign.values, color = '#e3b18d')
plt.title('Mean % subscription depending on number of contacts')
plt.xlabel('number of contacts bucket')
plt.ylabel('% subscription')
plt.show()

In [ ]:
df_new[df_new['campaign_buckets'] == 2]['campaign_cleaned'].min()

*Observations:*
* From the diagram above we can conclude, that marketing campaigns should concentrate on customers with account balance greater than $1362.
* Subscrpition rate tends to be higher for customers below 32 years old or above 54 years old.
* From the plot above we see that average take up rate is below 50% if the number of contacts during the campaign exceeds 4.

________

### Recommendations

I recommend the following based on analysis conducted:

* Contact between 750-1000 customers per month. Months (Mar, Sep, Oct & Dec) when less customers were contacted resulted in higher subscription rate.
* Customers should have minimum $1350 balance in their account.
* Include customers who are under 35 or over 55 years of age.
* Customers who are not married.
* Maximum number of calls per customer each month should not exceed 4. 
* Customers should not have credit default or have housing/personal loans.
* Also prioritse calling customers who either have not been contacted previously or had a successful outcome from previous campaign.

________